# Step 1: Extract Relevant BlueCruise Trip Data (Complete Trip Data - i.e Longer Time Series)

In [1]:
# Imports
import pandas as pd
from google.cloud import bigquery
import os

## GCP Setup:

In [4]:
# # Google Cloud Authentication: 
assert os.system('gcloud auth application-default login --quiet') == 0

In [5]:
# # Insert your project ID here:
PROJECT_ID = "ford-5bba11084fd31e17ec109f0c"

In [6]:
assert os.system(f"gcloud config set core/project {PROJECT_ID}") == 0

In [7]:
!gcloud config list

[core]
disable_usage_reporting = True
project = ford-5bba11084fd31e17ec109f0c



Your active configuration is: [default]


## Query: Extract BlueCruise Data from BigQuery (GCP)
- Initial Features extracted are:
    - trip_id
    - date_time
    - veh_long_vel_mps
    - veh_ltrl_vel_mps
    - veh_yaw_rate_radps
    - veh_accel_mps2
    - veh_jerk_mps3


In [ ]:
q = """
SELECT
/* Identifiers */
dpefa_bc_007_trip_d as trip_id,
 
/*Time*/
dpefa_bc_007_event_local_m as date_time,
 
/* Driving Characteristics */
dpefa_bc_007_hst_veh_long_vlcy_r as veh_long_vel_mps,
dpefa_bc_007_hst_veh_ltrl_vlcy_r as veh_ltrl_vel_mps,
dpefa_bc_007_hst_veh_yaw_rate_r as veh_yaw_rate_radps,
dpefa_bc_007_acc_mps2 as veh_accel_mps2,
dpefa_bc_007_jerk_mps3 as veh_jerk_mps3,
 
 
FROM `prj-dfad-31-usrda-p-31.dlobo1_bluecruise.gdpefa_adas_bc_ada_lm_vw` /* Change this to your dataset */
WHERE EXTRACT(date from dpefa_bc_007_event_m) between '2023-7-1' AND '2023-8-1' /*Set Dates of Interest*/
AND dpefa_bc_007_vin_17_x IN (SELECT
  dpefa_bc_007_vin_17_x AS vin,
  FROM `prj-dfad-31-usrda-p-31.dlobo1_bluecruise.gdpefa_adas_bc_ada_lm_vw`
  WHERE dpefa_bc_007_vin_17_x IN (
    SELECT DISTINCT bc.dpefa_bc_007_vin_17_x
    FROM `prj-dfad-31-usrda-p-31.dlobo1_bluecruise.gdpefa_adas_bc_ada_lm_vw` AS bc
    INNER JOIN `ford-5bba11084fd31e17ec109f0c.Ford_Pro_09112023.management_lease_summary_09112023` AS mgmt_lease
    ON bc.dpefa_bc_007_vin_17_x = mgmt_lease.VIN
    WHERE bc.dpefa_bc_007_event_m BETWEEN '2023-7-1' AND '2023-8-1' /* SET DATES OF INTEREST */
  )
  AND dpefa_bc_007_event_m BETWEEN '2023-7-1' AND '2023-8-1' /* SET DATES OF INTEREST */
  GROUP BY dpefa_bc_007_vin_17_x
  ORDER BY COUNT(dpefa_bc_007_vin_17_x) DESC
  LIMIT 10) /* SET NUMBER OF VINS OF INTEREST (Top X number of Vins with most BC data) */
"""

In [9]:
# Excecute the query
client = bigquery.Client()
df = client.query(q)
df_bc = df.to_dataframe()

## Data Preprocessing
- Change date time object into ISO 8601 format
- Filter out trips that are shorter than 12 seconds long [Arbitrary duration -> Can tune]

In [ ]:
# Convert the date_time column to datetime format
df_bc['date_time'] = pd.to_datetime(df_bc['date_time'], format='ISO8601')

# Find the start and end time of each trip and the duration of each trip
start_time = df_bc.groupby('trip_id')['date_time'].first()
end_time = df_bc.groupby('trip_id')['date_time'].last()
trip_duration = end_time - start_time

# Create a new dataframe with the trip number, start time, end time, and duration
trip_info = pd.DataFrame({'start_time': start_time, 'end_time': end_time, 'duration': trip_duration})
trip_info.reset_index(inplace=True)
trip_info.sort_values('duration', ascending=False, inplace=True) # sort the dataframe by longest trip first

# Remove all trips from df_bc that have a trip length less than 12 seconds using the trip_info DataFrame
valid_trip_nums = trip_info[trip_info['duration'] >= pd.Timedelta(seconds=12)]['trip_id'].unique()
df_bc = df_bc[df_bc['trip_id'].isin(valid_trip_nums)]

## Save Dataframe

In [11]:
# Create data folder if it doesn't exist
if not os.path.exists('data'):
    os.makedirs('data')

# Saving the df to a csv file [Rename this to include the date range]
df_bc.to_csv('data/df_bc_trips_PLACEHOLDER.csv', index = False)